# Core: 6. Context serialization

This tutorial shows context serialization.
First of all, let's do all the necessary imports from Chatsky.

In [1]:
# installing dependencies
%pip install -q chatsky


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
import logging

from chatsky.script import TRANSITIONS, RESPONSE, Context, Message
import chatsky.script.conditions as cnd

from chatsky.pipeline import Pipeline
from chatsky.utils.testing.common import (
    check_happy_path,
    is_interactive_mode,
    run_interactive_mode,
)

This function returns the user request number.

In [3]:
def response_handler(ctx: Context, _: Pipeline) -> Message:
    return Message(f"answer {len(ctx.requests)}")

In [4]:
# a dialog script
toy_script = {
    "flow_start": {
        "node_start": {
            RESPONSE: response_handler,
            TRANSITIONS: {("flow_start", "node_start"): cnd.true()},
        }
    }
}

# testing
happy_path = (
    ("hi", "answer 1"),
    ("how are you?", "answer 2"),
    ("ok", "answer 3"),
    ("good", "answer 4"),
)

Draft function that performs serialization.

In [5]:
def process_response(ctx: Context):
    ctx_json = ctx.model_dump_json()
    if isinstance(ctx_json, str):
        logging.info("context serialized to json str")
    else:
        raise Exception(f"ctx={ctx_json} has to be serialized to json string")

    ctx_dict = ctx.model_dump()
    if isinstance(ctx_dict, dict):
        logging.info("context serialized to dict")
    else:
        raise Exception(f"ctx={ctx_dict} has to be serialized to dict")

    if not isinstance(ctx, Context):
        raise Exception(f"ctx={ctx} has to have Context type")

In [6]:
pipeline = Pipeline.from_script(
    toy_script,
    start_label=("flow_start", "node_start"),
    post_services=[process_response],
)

if __name__ == "__main__":
    check_happy_path(pipeline, happy_path)
    if is_interactive_mode():
        run_interactive_mode(pipeline)

(user) >>> text='hi'
 (bot) <<< text='answer 1'
(user) >>> text='how are you?'
 (bot) <<< text='answer 2'
(user) >>> text='ok'
 (bot) <<< text='answer 3'
(user) >>> text='good'
 (bot) <<< text='answer 4'
